### Study to highlight the relationship between the existancede of PCS, the volume of the brain and the number of voxel.

You need BrainVisa to run this code. 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from soma import aims

ModuleNotFoundError: No module named 'soma'

#### Load the labels for the right and the left PCS on the ACCP dataset

In [ ]:
labeled_ACCP = pd.read_csv("/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/subjects_labels.csv")
labeled_ACCP = labeled_ACCP[['long_name','Left_PCS', 'Right_PCS']]

def encoder(df, columns):
    code = {'present':1,
            'absent':0}
    for col in columns:
        df[col] = df[col].map(code)
    return df

encoder(labeled_ACCP,['Left_PCS', 'Right_PCS'])
labeled_ACCP['long_name'].dtype

#### Let's do it on both hemispheres at the same time

In [ ]:
raw_skeleton_path = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/skeletons/raw/'
two_mm_skeleton_path = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/skeletons/2mm/'
list_path = [raw_skeleton_path,two_mm_skeleton_path]

In [ ]:
vol = aims.read(f'{raw_skeleton_path}R/Rskeleton_generated_sub-inhibition16_ses-pretraining_T1w.nii.gz')
print(vol)

In [ ]:
nb_vox = vol.np
print(nb_vox.shape)
print(np.count_nonzero(nb_vox))

In [ ]:
dic = {}

for repo_path in list_path:
    for side in ['R', 'L']:
        nb_vox_dic = {}
        path = f"{repo_path}/{side}/"
        for file in os.listdir(path):
            if file.endswith('.nii.gz'):
                #print(file)
                #print(file[20:-7]) # to get the name of the subject
                vol = aims.read(f'{path}{file}')
                nb_vox = vol.np
                nb_vox_dic[file[20:-7]]= np.count_nonzero(nb_vox) # calculate the number of voxel that are not zeros
        dic[f'nb_vox_{side}_{repo_path[-4:-1]}'] = nb_vox_dic

In [ ]:
for dicki in dic:
    df = pd.DataFrame({'long_name': list(dic[dicki].keys()), dicki: list(dic[dicki].values())})
    labeled_ACCP  = labeled_ACCP.merge(df, on='long_name', how='inner')

In [ ]:
labeled_ACCP['nb_vox_tot_raw'] = labeled_ACCP['nb_vox_R_raw'] + labeled_ACCP['nb_vox_L_raw']
labeled_ACCP['nb_vox_tot_2mm'] = labeled_ACCP['nb_vox_R_2mm'] + labeled_ACCP['nb_vox_L_2mm']
labeled_ACCP

In [ ]:
sns.displot(labeled_ACCP, x='nb_vox_tot_raw', hue='Left_PCS', bins=30, stat="density", common_norm=False)
plt.show()

In [ ]:
sns.displot(labeled_ACCP, x='nb_vox_tot_2mm', hue='Left_PCS', bins=30, stat="density", common_norm=False)
plt.show()

In [ ]:
sns.displot(labeled_ACCP, x='nb_vox_tot_raw', hue='Right_PCS', bins=30, stat="density", common_norm=False)
plt.show()

In [ ]:
sns.displot(labeled_ACCP, x='nb_vox_tot_2mm', hue='Right_PCS', bins=30, stat="density", common_norm=False)
plt.show()